In [5]:
#Goal is to find the worst performing products
import numpy as np

#import dataset
import pandas as pd
df = pd.read_excel("North Central Sales 6 Months GIT v2.xlsx", sheet_name="North Central")

#remove whitespace from column names for easy access
df.columns = [c.replace(' ', '_') for c in df.columns]
df.columns = [c.replace('$', 'usd') for c in df.columns]
print(df.columns)

#Get number of customers that buy from Packaging, From facilities and from both
customers = df.LEGACY_CUSTOMER_CD.unique()
customer_to_seg = {}
for c in customers:
    customer_to_seg[c] = []

for c, s in zip(df.LEGACY_CUSTOMER_CD.values, df.segment.values):
    if s not in customer_to_seg[c]:
        customer_to_seg[c].append(s)


pack_count = 0
facility_count = 0
both_count = 0


for c, l in customer_to_seg.items():
    if "Packaging" in l and "Facility Solutions" not in l:
        pack_count += 1
    if "Facility Solutions" in l and "Packaging" not in l:
        facility_count += 1
    if "Packaging" in l and "Facility Solutions" in l:
        both_count += 1
print("Total Number of Customers:", len(customers), "\n")
print("Only Packaging number of customers:", pack_count)
print("Only Facilities number of customers:", facility_count)
print("Number of Customers that buy from both:", both_count)

Index(['legacy_system_cd', 'legacy_division_cd', 'legacy_product_cd',
       'legacy_product_desc', 'core_item_flag', 'segment', 'PROD_CAT_1_NAME',
       'PROD_CAT_2_NAME', 'PROD_CAT_3_NAME', 'PROD_CAT_4_NAME',
       'legacy_vendor_cd', 'stocking_flag', 'LEGACY_CUSTOMER_CD',
       'saalfeld_customer_flag', 'national_acct_flag', 'ship-to_zip_code',
       'sales_channel', 'qty_6mos', 'cogs_6mos', 'Sales_6_mos', 'picks_6mos',
       'Margin_%', 'net_OH', 'net_OH_usd', 'pallet_quantity', 'item_poi_days',
       'DIOH'],
      dtype='object')
Total Number of Customers: 3469 

Only Packaging number of customers: 2043
Only Facilities number of customers: 644
Number of Customers that buy from both: 782


In [6]:
#get the important products
imp_customers = df[df.national_acct_flag == "Y"]
important_products = imp_customers.legacy_product_cd.unique()
print(len(important_products))

products = df.legacy_product_cd.unique()
#print(len(products))

#Create a dictionary of products to sales
prod_to_sales = {}
for p in products:
    prod_to_sales[p] = 0

    
for p,s in zip(df.legacy_product_cd.values, df.Sales_6_mos.values):
    prod_to_sales[p] += s


#get the total value of sales
total_sales = np.nansum(list(prod_to_sales.values()))

#Create a dictionary of products to cogs
prod_to_cogs = {}
for p in products:
    prod_to_cogs[p] = 0

for p, c in zip(df.legacy_product_cd.values, df.cogs_6mos.values):
    prod_to_cogs[p] += c

#create a dictionary of products to profit = sales - cogs
prod_to_profit = {}
for p in products:
    prod_to_profit[p] = 0

for p in products:
    prod_to_profit[p] += prod_to_sales[p] - prod_to_cogs[p]

#get the total profit
total_profit = np.nansum(list(prod_to_profit.values()))

6547


In [7]:
#Create a dictionary of products to total number of picks for that product
prod_to_picks = {}
for p in products:
    prod_to_picks[p] = 0

for p,pk in zip(df.legacy_product_cd.values, df.picks_6mos.values):
    prod_to_picks[p] += pk

In [8]:
#See how many products are profitable and how many are losing money
positive = {}
for x,y in prod_to_profit.items():
    if y >= 0:
        positive[x] = y

print("Number of Profitable products: ", len(positive.values()))
print("Total number of products: ", len(prod_to_profit.values()))

Number of Profitable products:  15525
Total number of products:  16035


In [18]:
#Create a dictionary of product -> number of warehouses where it is stored in one region
prod_to_warehouse = {}
for p in products:
    prod_to_warehouse[p] = []

for p, w in zip(df.legacy_product_cd.values, df.legacy_division_cd):
    if w not in prod_to_warehouse[p]:
        prod_to_warehouse[p].append(w)

prod_to_nw = {}
for p in products:
    prod_to_nw[p] = len(prod_to_warehouse[p])

print("Number of products in only one warehouse:", list(prod_to_nw.values()).count(1))
print("Number of products in two warehouses:", list(prod_to_nw.values()).count(2))
print("Number of products in three warehouses:", list(prod_to_nw.values()).count(3))
print("Number of products in four warehouses:", list(prod_to_nw.values()).count(4))
print("Total number of products:", len(products))

#MAKE A HISTOGRAM OF NUMBER OF WAREHOUSES THAT STORE A PARTICULAR PRODUCT

Number of products in only one warehouse: 12383
Number of products in two warehouses: 1651
Number of products in three warehouses: 729
Number of products in four warehouses: 383
sum 15146
Total number of products: 16035


In [2]:
#Create an api request of travel distances
import numpy as np

#import dataset
import pandas as pd
loc_df = pd.read_excel("locations.xlsx", sheet_name="Sheet1")
#remove whitespace from column names for easy access
loc_df.columns = [c.replace(' ', '_') for c in loc_df.columns]
print(loc_df.columns)

Index(['legacy_division_cd', 'legacy_division_desc', 'city', 'STATE',
       'zipcode'],
      dtype='object')


In [ ]:
#Compare the quantity stored in each warehouse to the Net On Hand Inventory in Quantity 